# Stability of the AMOC

In [ ]:
import os
import sys
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
sys.path.append("..")
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

## $M_{ov}$ calculation

The fundamental idea behind the $M_{ov}$ parameter is the effect of a MOC shutdown has on the salinity in the North Atlantic and hence NADW formation.
If the overturning circulation imports freshwater to the Atlantic, a MOC shutdown will increase salinity which counteracts the shutdown.
Vice versa, a freshwater export due to the MOC will lead to freshening of the North Atlantic in case of a shutdown and hence a strengthening of the shut down state.

The Atlantic freshwater budget in steady state is

$$\left[E - P - R \right] = M_{ov} + M_{az} + M_{dif} + M_{BS}$$

with evaporation $E$, precipitation $P$, runoff $R$, as well as freshwater transport terms due to the overturning circulation $M_{ov}$, the azonal gyre circulation $M_{az}$, diffusion $M_{dif}$, and exchange via the Bering Strait $M_{BS}$.


The overturning freswater transport term $M_{ov}$ is given differently in different publications:

_de Vries et al. (2005)_:

$$M_{ov} = - \frac{1}{S_0} \int \bar{v}(z) \left[ \langle S(z) \rangle - S_0 \right] \,\mathrm{d}z$$

where $\bar{v}(z) = \int v(x,z) \,\mathrm{d}x$ is the zonal integral and $\langle S \rangle (z) = \int S(x,z) \,\mathrm{d}x \, \big/ \int \mathrm{d}x$ is the zonal mean, such that

\begin{align}
M_{ov} & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x} - S_0 \right] \,\mathrm{d}z \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x} \right] \,\mathrm{d}z + \int \int v(x,z) \, \mathrm{d}x \mathrm{d}z  \tag{1}
\end{align}

_Mecking et al. (2017)_:

$$M_{ov} = - \frac{1}{S_0} \int \int v^*(z) \langle S(z) \rangle \, \mathrm{d}x \mathrm{d}z$$

where $v^*(z) = \langle v \rangle (z) - \bar{v} = \int v(x,z) \,\mathrm{d}x \, \big/ \int \mathrm{d}x - \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z \, \big/ \int \int \mathrm{d}x \mathrm{d}z$ such that

\begin{align}
M_{ov} & = - \frac{1}{S_0} \int \int \left[ \frac{\int v(x,z) \,\mathrm{d}x}{ \int \mathrm{d}x} - \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \int \mathrm{d}x \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}x \mathrm{d}z \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x - \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \frac{1}{S_0} \int  \left[ \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z \\
  & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \frac{1}{S_0} \int \left[ \frac{ \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z  \times   \int S(x,z) \,\mathrm{d}x }{\int \int \mathrm{d}x \mathrm{d}z } \right] \, \mathrm{d}z \\
  & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \left[ \frac{1}{S_0} \frac{  \int \int S(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \int \mathrm{d}x \mathrm{d}z } \right] \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z  \tag{2}
\end{align}

Equations (2) and (1) are equal if the reference salinity is equal to the section average: $S_0 = \int \int S(x,z) \,\mathrm{d}x \mathrm{d}z \, \big/ \int \int \mathrm{d}x \mathrm{d}z $

In [ ]:
from paths import file_ex_ocn_ctrl, file_ex_ocn_lpd, path_prace

In [ ]:
%%time
j_34 = 810               # latitude number at the Cape of Good Hope
i_SA, i_CGH = 555, 1299  # longitude numbers of South America and the Cape of Good Hope
mf_fn = '/projects/0/prace_imau/prace_2013081679/cesm1_0_4/spinup_pd_maxcores_f05_t12/OUTPUT/ocn/hist/monthly/spinup_pd_maxcores_f05_t12.pop.h.0*.nc'
ds = xr.open_mfdataset(mf_fn, decode_times=False, combine='by_coords',
                       concat_dim='time', compat='override', parallel=True)[['SALT', 'VVEL', 'DXT', 'DXU', 'z_t']].isel({'time':0, 'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
# ds_lpd  = xr.open_dataset(file_ex_ocn_lpd , decode_times=False)

In [ ]:
ds

In [ ]:
ds.SALT.plot()
plt.tight_layout()

In [ ]:
S0 = (ds.SALT*ds.DXT*ds.z_t).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.SALT>0)*ds.z_t).sum(dim=['nlon','z_t'])
L = ds.DXT.sum(dim='nlon')
print(f'S_0 = {S0.values:5.2f} g/kg\nL   = {L.values/1e5:5.0f} km')

In [ ]:
(ds.VVEL*ds.DXU).sum('nlon').plot()

In [ ]:
( (ds.VVEL*ds.DXU).sum('nlon') * ds.SALT.mean(dim='nlon', skipna=True)).plot()

In [ ]:
((ds.VVEL*ds.DXU).sum('nlon')).plot()

In [ ]:
(( (ds.VVEL*ds.DXU).sum('nlon') * ds.SALT.mean('nlon', skipna=True) )*ds.z_t).plot()

In [ ]:
( ds.VVEL*ds.DXU*(ds.SALT.mean('nlon', skipna=True)-S0) ).plot()

In [ ]:
((ds.SALT.mean('nlon', skipna=True)-S0)).plot()
plt.axhline()

In [ ]:
# function given by de Vries et al
Mov = ( -1/S0*(( ds.VVEL*ds.DXU*(ds.SALT.mean('nlon', skipna=True)-S0) )*ds.z_t).sum(['nlon', 'z_t'])  )/ 1e12  # 1e12 cm^3/s = 1 Sv
print(Mov.values)

In [ ]:
# can use .mean function because at 34S the grid is rectilinear
Mov = ( -1/S0*(( (ds.VVEL*ds.DXU).sum('nlon') * ds.SALT.mean('nlon', skipna=True) )*ds.z_t).sum('z_t') \
      + (ds.VVEL*ds.z_t*ds.DXU).sum(['nlon','z_t']) ) / 1e12  # 1e12 cm^3/s = 1 Sv
print(Mov.values)

In [ ]:
((ds.VVEL*ds.DXU).sum('nlon') * (ds.SALT.mean('nlon', skipna=True)-S0) ).plot()

In [ ]:
ds

## Salinity bias
_Mecking et al. (2017)_ point out that the CMIP5 $M_{ov}$ values are influenced by salinity biases, so it is of interest to quantify these in the CESM simulations.